starting out like the other notebooks ;-)

In [1]:
from cffi import FFI
ffi = FFI()

#### for a first connection to the alpha2 network we only need the FfiResult-Struct (and a fake-definition for the authenticator)

In [2]:
ffi.cdef('''
typedef int int32_t;
typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct {
    void* core_tx;
    void* _core_joiner;
} Authenticator;
''')

and for our connection test the stem function + login/create account functions

In [3]:
ffi.cdef('''
void app_exe_file_stem(void* user_data, void(*)(void*, FfiResult*, char*));
void login(char* account_locator, char* account_password, void* user_data, void(*)(void*), void(*)(void*, FfiResult*, Authenticator*));
void create_acc(char* account_locator, char* account_password, char* invitation, void* user_data, void(*)(void*), void(*)(void*, FfiResult*, Authenticator*));
''')

In [4]:
class Auth:
    def __init__(self,
            name='noAuth',
            version='0.0.0',
            vendor='rid',
            libLocation= '../compiled_binaries/libsafe_authenticator.so',
            addr='http://localhost'):
        self.name = name
        self.version = version
        self.vendor = vendor
        self.url = addr
        self.lib = ffi.dlopen(libLocation)
        
    def defaultFfiResult(self, result, actionDescription):
    
        if result.error_code == 0:
            print('successfully ' + actionDescription)
        else:
            print('an Error occured - Error Code: ' + str(result.error_code))
            print('Error description: ' + str(ffi.string(result.description)))
            
    def toByteIfString(self, parameter, encoding):
        if type(parameter) == str:
            return parameter.encode()
        else:
            return parameter

#### the login function needs to know at least the two first arguments to work

In [5]:
def login(self,account_locator,account_password,user_data=None,disconnect_notifier_cb=None,cb=None,encoding='utf-8'):
    ''' string/bytes, string/bytes, [any], [function], [function], [encoding]
        char* account_locator, char* account_password, void* user_data
        
        > return values of the callback functions:
        disconnect_notifier_cb - void* user_data
        cb - void* user_data, FfiResult* result, Authenticator* authenticator
    ''' 
    
    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        
        if disconnect_notifier_cb:
            disconnect_notifier_cb(user_data)
        else:
            pass
    
    @ffi.callback("void(void*,FfiResult*,Authenticator*)")
    def o_cb(user_data,result,authenticator):
        
        if cb:
            cb(user_data,result,authenticator)
        else:
            self.defaultFfiResult(result,'logged into the SAFE Network')
    
    account_locator = ffi.new('char[]',self.toByteIfString(account_locator,encoding))
    password = ffi.new('char[]',self.toByteIfString(account_password,encoding))
    if user_data:
        userData = ffi.new_handle(user_data)
    else:
        userData = ffi.NULL
    
    self.lib.login(account_locator,password,userData,o_disconnect_notifier_cb,o_cb)
    
    
Auth.login = login
del(login)

parameters for account creation

account_locator | account_password | invitation | user_data
--- | --- | --- | ---
important; your first login string | important; your second login string | random string in mock routing | no clue


In [6]:
def create_acc(self,account_locator,account_password,invitation,user_data=None,disconnect_notifier_cb=None,cb=None,encoding='utf-8'):
    ''' string/bytes, string/bytes, string/bytes, [any], [function], [function], [encoding]
        char* account_locator, char* account_password, char* invitation, void* user_data
        
        > return values of the callback functions:
        disconnect_notifier_cb - void* user_data
        cb - void* user_data, FfiResult* result, Authenticator* authenticator
    ''' 
    
    @ffi.callback("void(void*)")
    def o_disconnect_notifier_cb(user_data):
        
        if disconnect_notifier_cb:
            disconnect_notifier_cb(user_data)
        else:
            pass
    
    
    @ffi.callback("void(void*,FfiResult*,Authenticator*)")
    def o_cb(user_data,result,authenticator):
        
        if cb:
            cb(user_data,result,authenticator)
        else:
            self.defaultFfiResult(result,'created new SAFE Network Account')
    
    account_locator = ffi.new('char[]',self.toByteIfString(account_locator,encoding))
    password = ffi.new('char[]',self.toByteIfString(account_password,encoding))
    invitation = ffi.new('char[]',self.toByteIfString(invitation,encoding))
    if user_data:
        userData = ffi.new_handle(user_data)
    else:
        userData = ffi.NULL
    
    self.lib.create_acc(account_locator,password,invitation,userData,o_disconnect_notifier_cb,o_cb)
    
    
Auth.create_acc = create_acc
del(create_acc)

since the stem-function uses a callback-function to give back our crust stem name here we define one that will print the resulting name

In [7]:
userData=None
myResult=None


@ffi.callback("void(void*,FfiResult*, char*)")
def stem(user_data, result, name):
    global userData, myResult
    
    userData = user_data
    myResult=result
    print(ffi.string(name))

---

In [8]:
au=Auth(libLocation='../compiled_binaries/libsafe_app.so')

---

In [9]:
au.lib.app_exe_file_stem(ffi.NULL,stem)

b'python3'


---

so i would assume our crust config needs to be called [python3.crust.config in reference to nikitas comment in the forum](https://forum.safedev.org/t/how-to-use-safe-client-libs-in-python/1029/38)

I did add the file in the folder where the \*.so-library is (and i tried adding it to the script folder here too) but seems like i'm still doing something wrong ... 

In [10]:
my=Auth()

In [11]:
import getpass

username=getpass.getpass()

key=getpass.getpass()

my.login(username,key)

········
········
an Error occured - Error Code: -2000
Error description: b'Core error: Unexpected: Could not connect to the SAFE Network'
